<a href="https://colab.research.google.com/github/rubuntu/Taller_Introduccion_a_Ciencia_de_Datos_IA_e_Ingenieria_de_Datos/blob/main/sesion_07_introduccion_a_xgboost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sesión 07: Introducción a XGBoost

## Objetivos
- Comprender qué es XGBoost y por qué es potente en clasificación tabular.
- Preparar el dataset de default de tarjetas de crédito.
- Entrenar un primer modelo de XGBoost y evaluarlo.

---

## 🌳 Árboles de decisión

Un **árbol de decisión** es un modelo de machine learning que divide los datos en ramas y hojas para hacer predicciones.

* Se construye haciendo **preguntas secuenciales** sobre las variables (features).
* Cada nodo interno representa una condición (ej. *“¿edad < 30?”*).
* Cada rama representa una respuesta (*sí / no*).
* Cada hoja contiene un resultado (predicción de clase o valor).

📌 **Ventajas**

* Fácil de interpretar y visualizar.
* Maneja variables numéricas y categóricas.
* No requiere mucha preparación de datos.

📌 **Desventajas**

* Puede sobreajustar (*overfitting*).
* Sensibles a pequeñas variaciones en los datos.

---

## ⚡ Boosting

El **boosting** es una técnica de **ensemble learning** (modelos combinados) que mejora el rendimiento de modelos débiles (como árboles poco profundos).

La idea principal es:

1. Entrenar un **primer árbol**.
2. Ver los **errores** que cometió.
3. Entrenar un **segundo árbol** que se enfoque en corregir esos errores.
4. Repetir el proceso, combinando muchos árboles **débiles** para crear un **modelo fuerte**.

### Ejemplos de algoritmos de boosting:

* **AdaBoost**: ajusta pesos a las observaciones mal clasificadas.
* **Gradient Boosting**: corrige errores minimizando una función de pérdida (más preciso que AdaBoost).
* **XGBoost, LightGBM, CatBoost**: implementaciones modernas, rápidas y optimizadas de gradient boosting.

📌 **Ventajas**

* Suele lograr alta precisión.
* Maneja relaciones no lineales y complejas.
* Flexible (puede usarse para regresión y clasificación).

📌 **Desventajas**

* Más difícil de interpretar que un solo árbol.
* Computacionalmente más costoso.
* Puede sobreajustar si no se ajustan bien los hiperparámetros.

---

👉 En pocas palabras:

* **Árbol de decisión** = un modelo sencillo y visual.
* **Boosting** = muchos árboles pequeños que se ayudan entre sí corrigiendo errores.

---
## 🔹  ¿Qué es XGBoost?
- **Extreme Gradient Boosting**: algoritmo basado en árboles de decisión y boosting.
- Combina múltiples árboles débiles en un modelo fuerte.
- Ventajas:
  - Maneja bien datos tabulares.
  - Regularización integrada (evita overfitting).
  - Muy eficiente y escalable.

---



## 🔹 ¿Qué son los SHAP Values?

Los **SHAP Values** (**SHapley Additive exPlanations**) son una técnica de **explicabilidad de modelos de machine learning**.
Su objetivo es responder a la pregunta:

👉 *“¿Cuánto aporta cada variable (feature) a la predicción de un modelo para un caso específico?”*

Es decir, no solo dicen qué variables son importantes en general, sino también **cómo influyen en la predicción de cada individuo** (ejemplo: por qué el cliente 123 fue clasificado como riesgo alto).

Cada predicción se descompone como:

$$
\text{Predicción} = \text{Valor base (esperado)} + \sum_{i=1}^M \text{SHAP value}_i
$$

* **Valor base**: la predicción promedio si no supiéramos nada (ej. probabilidad media de default en el dataset).
* **SHAP value de una feature**: cuánto sube o baja la predicción al considerar esa variable en el caso concreto.

---

## 🔹 Origen: Teoría de Juegos de Shapley (1953)

Los SHAP Values se inspiran en los **valores de Shapley**, un concepto de la **teoría de juegos cooperativos** de Lloyd Shapley (premio Nobel de Economía, 2012).

En un juego cooperativo:

* Hay **jugadores** que contribuyen a una recompensa común.
* Los **valores de Shapley** determinan una forma justa de repartir la recompensa según el aporte de cada jugador.

En Machine Learning:

* Los **jugadores** son las **variables (features)**.
* La **recompensa** es la **predicción del modelo**.
* El **valor de Shapley** indica cuánto contribuyó cada variable a la predicción final.

---

## 🔹 Ventajas de SHAP

* **Explicaciones consistentes y justas** (garantiza propiedades matemáticas de equidad heredadas de la teoría de juegos).
* **Explicaciones locales y globales**:

  * Locales → por instancia (ej. cliente específico).
  * Globales → promedio de contribuciones, para ranking de importancia de variables.
* **Aplicable a cualquier modelo** (árboles, redes neuronales, regresiones, etc.).
* Tiene implementaciones optimizadas para modelos de árboles como **XGBoost, LightGBM, CatBoost**, lo que lo hace muy usado en datos tabulares.

---

✅ Resumen:
Los **SHAP values** son una adaptación de los **valores de Shapley (teoría de juegos de 1953)** al mundo del machine learning. Nos permiten interpretar modelos complejos asignando a cada variable un aporte justo y cuantitativo en cada predicción.

---


## Dataset
Fuente: [UCI Default of Credit Card Clients](https://archive.ics.uci.edu/ml/datasets/default+of+credit+card+clients)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/00350/default%20of%20credit%20card%20clients.xls"
df = pd.read_excel(url, header=1)

df.rename(columns={"default payment next month": "default"}, inplace=True)
df.head()

In [ ]:
df.describe().T

In [ ]:
df.nunique()

In [ ]:
df.hist(bins=20, figsize=(10, 6))

---

## Preprocesamiento

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X = df.drop(columns=["ID","default"])
y = df["default"]

feature_names = list(X.columns)

#scaler = StandardScaler()
#X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)


In [ ]:
# X_scaled es un array de NumPy
# stats = {
#    "count": X_scaled.shape[0],
#    "mean": np.mean(X_scaled, axis=0),
#    "std": np.std(X_scaled, axis=0, ddof=1),  # ddof=1 para que sea como pandas
#    "min": np.min(X_scaled, axis=0),
#    "25%": np.percentile(X_scaled, 25, axis=0),
#    "50%": np.median(X_scaled, axis=0),
#    "75%": np.percentile(X_scaled, 75, axis=0),
#    "max": np.max(X_scaled, axis=0)
#}
#pd.DataFrame(stats, index=X.columns)

---

## Entrenar XGBoost

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from xgboost import XGBClassifier, plot_importance, DMatrix
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix, roc_curve, auc

# Calcular el ratio
neg, pos = np.bincount(y_train)
scale = neg / pos
print(f"Negativos: {neg}, Positivos: {pos}, scale_pos_weight: {scale:.2f}")

# Modelo con ajuste por desbalance y regularización
xgb = XGBClassifier(
    n_estimators=200,
    learning_rate=0.1,
    max_depth=4,
    random_state=42,
    eval_metric="logloss",
    scale_pos_weight=scale,   # 👈 desbalance
    reg_lambda=1.0,           # 👈 regularización L2 (default=1.0)
    reg_alpha=0.1             # 👈 regularización L1 (default=0.0, aquí lo activamos)
)


xgb.fit(X_train, y_train)
y_pred = xgb.predict(X_test)
y_proba = xgb.predict_proba(X_test)[:,1]

print(classification_report(y_test, y_pred))
print("AUC:", roc_auc_score(y_test, y_proba))

## Explicación del código

---

### 1) Calcular el ratio de desbalance

```python
neg, pos = np.bincount(y_train)
scale = neg / pos
print(f"Negativos: {neg}, Positivos: {pos}, scale_pos_weight: {scale:.2f}")
```

* `np.bincount(y_train)` cuenta cuántas instancias hay de la clase **0** (*neg*) y de la clase **1** (*pos*).

  > Requisito: `y_train` debe ser binaria con etiquetas **0/1**. Si no lo es, conviértelas (p. ej., con `LabelEncoder`).
* `scale = neg / pos` calcula el **ratio** entre clases.

  * Si hay muchos más 0 que 1, `scale` será grande → se **aumenta el peso** de los positivos en el entrenamiento.
* Este valor se usa en `scale_pos_weight` (ver abajo).

**¿Por qué sirve?**
En datasets desbalanceados, el modelo puede “ignorar” la clase minoritaria. `scale_pos_weight` **repondera** la pérdida para que los positivos “pesen” más, ayudando a detectar la clase rara (p. ej., *default*).

---

### 2) Modelo con ajuste por desbalance y regularización

```python
xgb = XGBClassifier(
    n_estimators=200,
    learning_rate=0.1,
    max_depth=4,
    random_state=42,
    eval_metric="logloss",
    scale_pos_weight=scale,   # 👈 desbalance
    reg_lambda=1.0,           # 👈 L2 (default=1.0)
    reg_alpha=0.1             # 👈 L1 (default=0.0, aquí lo activamos)
)
```

* **`scale_pos_weight=scale`**: pondera más la **clase positiva (1)** en la función de pérdida. Regla práctica: `neg/pos`.

  * Útil cuando **pos** ≪ **neg**.
  * Alternativa: pasar **`sample_weight`** por instancia al `fit` si quieres pesos más finos.
* **Regularización**: controla la **complejidad** del ensemble para evitar **overfitting**.

  * `reg_lambda` (**L2**): reduce magnitudes de los pesos en las hojas → suaviza el modelo.
  * `reg_alpha` (**L1**): favorece **sparsidad** (muchos pesos = 0) → simplifica y puede mejorar interpretabilidad.
* Otros hiperparámetros (ya configurados):

  * `n_estimators=200` y `learning_rate=0.1`: más árboles + pasos pequeños → aprendizaje más estable.
  * `max_depth=4`: árboles poco profundos reducen el riesgo de sobreajuste en tabulares.
  * `eval_metric="logloss"`: pérdida logarítmica binaria para seguimiento del entrenamiento.

> Tip de práctica: afina `scale_pos_weight`, `reg_lambda`, `reg_alpha`, `max_depth`, `min_child_weight`, `subsample`, `colsample_bytree` con **validación cruzada** o **early stopping**.

---

### 3) Entrenamiento y predicción

```python
xgb.fit(X_train, y_train)
y_pred  = xgb.predict(X_test)
y_proba = xgb.predict_proba(X_test)[:,1]
```

* `fit`: entrena el ensemble de árboles.
* `predict`: devuelve etiquetas 0/1 usando umbral 0.5 por defecto.
* `predict_proba` (columna `[:,1]`): devuelve **probabilidades** de clase positiva. Útil para métricas umbral-dependientes, **ROC/PR**, *calibration*, y para ajustar el umbral según costos de negocio.

> Tip: En problemas desbalanceados, **no quedarse con el umbral 0.5**. Eliger uno que optimice una métrica objetivo (ej., **F1**, **Recall\@Precision**, **Expected cost**).

---

### 4) Evaluación

```python
print(classification_report(y_test, y_pred))
print("AUC:", roc_auc_score(y_test, y_proba))
```

* `classification_report`: precisión, recall y F1 por clase (basadas en **y\_pred** → dependen del **umbral**).
* `roc_auc_score`: AUC-ROC sobre **probabilidades** (**y\_proba**).

  * **AUC** no depende del umbral y mide la **capacidad discriminativa** global del modelo.
  * En banca, suele complementarse con **KS**, **Gini** y **PR AUC** si hay fuerte desbalance.

---

### Buenas prácticas y mejoras rápidas

1. **Early Stopping** (evita sobreajuste y ahorra cómputo):

```python
xgb.fit(
    X_train, y_train,
    eval_set=[(X_test, y_test)],
    early_stopping_rounds=50,      # detiene si no mejora
    verbose=False
)
```

2. **Búsqueda de umbral óptimo** (ejemplo con F1):

```python
from sklearn.metrics import f1_score
import numpy as np

probas = y_proba
ths = np.linspace(0.01, 0.99, 99)
f1s = [f1_score(y_test, (probas >= t).astype(int)) for t in ths]
best_t = ths[int(np.argmax(f1s))]
print(f"Mejor umbral por F1: {best_t:.2f}, F1={max(f1s):.3f}")
```

3. **Cuándo usar `scale_pos_weight`**

* Úsar cuando **pos** ≪ **neg**. Si hay **pesos por instancia** (costos diferentes), considera `sample_weight` en `fit`.
* Evitar **mezclar** `scale_pos_weight` con otra lógica de pesos que duplique el efecto (p. ej., `sample_weight` ya desbalanceado) salvo que lo hagas intencionalmente.

4. **Sanity checks**

* Verificar que `pos > 0` (si no, `neg/pos` explota).
* Si las etiquetas no son 0/1, normalizarlas antes de `bincount`.

---

### Resumen

* Se calcula el **ratio de clases** para informar al modelo del desbalance (`scale_pos_weight`).
* Se añade **regularización L1/L2** para **controlar complejidad** y **evitar overfitting**.
* Se evalúa con **métricas por probabilidad (AUC)** y por etiqueta (report), recordando que el **umbral** es clave en desbalance.
* Considerar **early stopping** y **búsqueda de umbral** para mejores resultados prácticos.


In [ ]:
# === Matriz de confusión ===
cm = confusion_matrix(y_test, y_pred)

plt.figure(figsize=(5,4))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", cbar=False)
plt.xlabel("Predicción")
plt.ylabel("Valor real")
plt.title("Matriz de confusión")
plt.show()

# === Curva ROC ===
fpr, tpr, thresholds = roc_curve(y_test, y_proba)
roc_auc = auc(fpr, tpr)

plt.figure(figsize=(6,5))
plt.plot(fpr, tpr, label=f"XGBClassifier (AUC = {roc_auc:.2f})")
plt.plot([0,1], [0,1], linestyle="--", color="gray")  # línea diagonal
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("Curva ROC")
plt.legend(loc="lower right")
plt.show()


In [ ]:
# =========================================================
# Importancia de variables (built-in de XGBoost)
# kinds disponibles: "weight", "gain", "cover", "total_gain", "total_cover"
# =========================================================

# Asignar nombres al booster (si X_train era ndarray)
booster = xgb.get_booster()
booster.feature_names = feature_names

plt.figure(figsize=(7,6))
plot_importance(xgb, importance_type="gain", max_num_features=20)
plt.title("Importancia de variables (gain) - XGBoost")
plt.show()


In [ ]:
# SHAP values vía XGBoost (pred_contribs=True) y gráfico de barras por mean |SHAP|
# - Usamos DMatrix para garantizar los nombres

dtest = DMatrix(X_test, feature_names=feature_names)
shap_values_full = booster.predict(dtest, pred_contribs=True)  # (n_samples, n_features + 1) incluye bias
shap_values = shap_values_full[:, :-1]  # quitamos bias

mean_abs_shap = np.mean(np.abs(shap_values), axis=0)
order = np.argsort(mean_abs_shap)[::-1]
top_k = min(20, len(feature_names))

plt.figure(figsize=(7,6))
plt.barh(np.array(feature_names)[order[:top_k]][::-1], mean_abs_shap[order[:top_k]][::-1])
plt.xlabel("Mean |SHAP value|")
plt.title("Importancia (media de |SHAP|) - Top features")
plt.tight_layout()
plt.show()

## 🔹 ¿Qué es un **Beeswarm plot** en SHAP?

El **Beeswarm plot** es el gráfico **resumen más usado en SHAP**.
Muestra, en **una sola figura**, cómo todas las variables influyen en las predicciones de un modelo.

Es una **visualización compacta y global** de los SHAP values.

---

## 🔹 ¿Cómo se interpreta?

1. **Eje Y (vertical):** lista de variables, ordenadas de arriba hacia abajo por **importancia global** (media del valor absoluto de SHAP).
2. **Eje X (horizontal):** valor SHAP → cuánto aporta la variable a la predicción.

   * Valores positivos → empujan la predicción hacia arriba (ej. mayor riesgo de default).
   * Valores negativos → empujan la predicción hacia abajo (ej. menor riesgo de default).
3. **Cada punto** = una observación (fila del dataset).
4. **Color del punto:** representa el valor real de la feature (bajo → azul, alto → rojo). Esto permite ver **patrones**:

   * Ejemplo: si para *edad*, los puntos rojos (edades altas) tienen SHAP negativo, significa que **edad alta disminuye el riesgo**.

---

## 🔹 ¿Por qué se llama “Beeswarm”?

Porque al juntar miles de puntos de distintas observaciones en una sola figura, parecen un **enjambre de abejas** alrededor de cada variable.
Los puntos se distribuyen horizontalmente para no superponerse, creando esa apariencia.

---

## 🔹 Ventajas del Beeswarm plot

* Resume toda la información en un solo gráfico.
* Permite ver **importancia global** y también **efectos locales** de cada variable.
* Muestra **no linealidades** y **dirección del impacto**.
* Mucho más informativo que un simple ranking de importancias.

---

✅ **En resumen:**
El **Beeswarm plot en SHAP** es una representación visual que combina importancia de variables y efecto individual de cada observación, mostrando tanto la magnitud como la dirección de la contribución de cada feature al modelo.

---


In [ ]:
import shap
import matplotlib.pyplot as plt

explainer = shap.TreeExplainer(xgb.get_booster())
shap_values_shap = explainer.shap_values(X_test)

# --- Beeswarm ---
shap.summary_plot(shap_values_shap, X_test, feature_names=feature_names, show=False)

plt.title("SHAP Values - Beeswarm", fontsize=14)  # 👈 título sobre el gráfico
plt.show()


## 🔹 ¿Qué es un *force plot* en SHAP?

El **force plot** es una de las visualizaciones clásicas de **SHAP (SHapley Additive exPlanations)**.
Sirve para mostrar, en una sola observación (una fila de tu dataset), **cómo cada variable contribuye a empujar la predicción** de tu modelo hacia arriba o hacia abajo respecto al valor base (*expected value*).

---

### 📌 Conceptos clave

* **Expected Value (valor base):**
  Es el valor promedio del modelo antes de ver ninguna característica (ejemplo: la probabilidad media de la clase positiva en el dataset).

* **SHAP values:**
  Para cada feature, indica si esa variable **aumenta** o **disminuye** la predicción respecto al expected value.

  * Valores SHAP positivos → empujan la predicción hacia la clase positiva (mayor probabilidad de default, churn, etc.).
  * Valores SHAP negativos → empujan hacia la clase negativa.

* **Predicción final:**
  Se obtiene sumando el expected value + la suma de todos los SHAP values de esa instancia.

---

### 🔎 ¿Cómo se interpreta el gráfico?

En el force plot:

* Hay una **barra horizontal** que representa el valor base al inicio y la predicción final al final.
* Cada feature aparece como una "fuerza":

  * **Rojo (→)**: empuja la predicción hacia arriba (hacia 1 en binario).
  * **Azul (←)**: empuja hacia abajo (hacia 0 en binario).

Es como una balanza de fuerzas: cada variable suma o resta hasta alcanzar la probabilidad final que da el modelo.

---

### 📊 Ejemplo práctico en tu caso (Bank Marketing)

Si la predicción es que un cliente **sí suscribirá un depósito**:

* Variables como `contact=cellular`, `month=mar`, `duration=longa llamada` pueden aparecer en rojo → empujando hacia **sí**.
* Variables como `age=young`, `previous=0`, `poutcome=unknown` pueden aparecer en azul → empujando hacia **no**.

Así podés ver **qué combinación específica de factores llevó a la decisión del modelo para ese cliente**.

---

👉 Resumen:
El **force plot** es una visualización **individual** que explica **cómo cada feature influyó en la predicción de un caso particular**, mostrando de manera gráfica las fuerzas que llevan desde el promedio (expected value) hasta la predicción final.

---


In [ ]:
## Force plot individual con SHAP (XGBoost)

# Elegimos un índice del test (ej. la primera fila)
i = 0

# Extraer valores de la fila y sus SHAP values
x_row = X_test.iloc[i]
shap_row = shap_values_shap[i]

# Inicializar visualización interactiva de SHAP
shap.initjs()

# Force plot para la instancia i
shap.force_plot(
    base_value=explainer.expected_value,  # valor base (expected value)
    shap_values=shap_row,                 # contribuciones SHAP de la fila
    features=x_row,                       # valores de las features de esa fila
    feature_names=feature_names,          # opcional si querés mostrar nombres explícitos
    matplotlib=True                      # interactivo en notebook (True = gráfico estático)
)

## 🔹 ¿Qué es un **Dependence Plot** en SHAP?

Un **Dependence Plot** es un gráfico que muestra **cómo influye una variable en las predicciones del modelo según sus valores**, usando los **SHAP values** como medida de impacto.

* En el eje **X** se representa el valor real de la **feature**.
* En el eje **Y** se representa el **SHAP value** de esa feature, es decir, cuánto aporta (positivo o negativo) al resultado del modelo.
* Cada punto es una observación del dataset (una fila).

👉 La interpretación es intuitiva:

* **Si los SHAP values son positivos** → esa variable aumenta la probabilidad/predicción del modelo.
* **Si los SHAP values son negativos** → esa variable reduce la predicción.

---

## 🔹 Características principales

1. **Relación variable-predicción**: muestra cómo los cambios en una feature modifican la salida del modelo.
2. **No linealidad**: a diferencia de modelos lineales, se pueden observar curvas, umbrales y patrones complejos.
3. **Coloración opcional**: muchas veces los puntos se colorean por otra feature correlacionada, lo que ayuda a descubrir **interacciones entre variables**.

---

## 🔹 Ejemplo práctico

Si analizamos un modelo de riesgo de crédito y hacemos un dependence plot de la variable **“ingresos”**:

* Eje X → valores de ingresos de los clientes.
* Eje Y → SHAP values de ingresos.
* Si observamos que **a bajos ingresos los SHAP values son positivos**, significa que bajos ingresos **aumentan la probabilidad de default** en la predicción del modelo.

---

✅ Resumen:
Un **Dependence Plot** con SHAP es como una **“curva de efecto”** que explica cómo el modelo utiliza una variable para tomar decisiones, revelando patrones e interacciones que un simple ranking de importancia no muestra.

---

In [ ]:
# Dependence plot de la feature más importante
top_feat_idx = order[0]
if hasattr(X_test, "iloc"):
    x_vals = X_test.iloc[:, top_feat_idx].to_numpy()
else:
    x_vals = X_test[:, top_feat_idx]

plt.figure(figsize=(6,5))
plt.scatter(x_vals, shap_values[:, top_feat_idx], s=12, alpha=0.6)
plt.xlabel(feature_names[top_feat_idx])
plt.ylabel("SHAP value")
plt.title(f"Dependence plot (SHAP) - {feature_names[top_feat_idx]}")
plt.tight_layout()
plt.show()

---

## 5. Preguntas de Discusión

### 1. ¿Qué ventajas ofrece XGBoost frente a modelos lineales en datos tabulares?

* **Relaciones no lineales:** mientras que los modelos lineales (como regresión logística) asumen una relación lineal entre variables y resultado, XGBoost puede capturar interacciones complejas y no lineales.
* **Ingeniería de características implícita:** los árboles crean automáticamente combinaciones y umbrales de variables, sin necesidad de transformar mucho los datos de entrada.
* **Manejo de valores atípicos y escalamiento:** no requiere normalización y es más robusto frente a outliers.
* **Regularización incorporada:** incluye parámetros de penalización (`lambda`, `alpha`) para controlar la complejidad y reducir el riesgo de sobreajuste.
* **Escalabilidad y velocidad:** está optimizado en C++ con paralelización y técnicas como “tree boosting” y “column block structure”, lo que lo hace muy eficiente para datasets grandes.
* **Importancia de variables y explicabilidad:** provee métricas de importancia y puede complementarse fácilmente con SHAP values para interpretar los modelos.

---

### 2. ¿Cómo influye el hiperparámetro `max_depth` en el overfitting?

* `max_depth` controla la **profundidad máxima de los árboles** en el ensemble.
* **Profundidad baja (ej. 2–4):**

  * Los árboles son simples, con menor riesgo de sobreajuste.
  * El modelo puede quedarse corto y subestimar patrones (underfitting).
* **Profundidad alta (ej. 8–15):**

  * Los árboles capturan muchos detalles, incluso ruido.
  * Aumenta el riesgo de overfitting, sobre todo en datasets pequeños o con ruido.
* En la práctica:

  * Se combina con otros parámetros (`min_child_weight`, `subsample`, `colsample_bytree`) para equilibrar sesgo y varianza.
  * Lo habitual en datos tabulares está entre 3 y 6.

---

### 3. ¿Qué métricas usarías en un banco para evaluar un modelo de default?

En problemas de riesgo crediticio, el costo de un error no es simétrico: **dar crédito a alguien que incumple es mucho más costoso** que rechazar un buen cliente. Por eso se usan varias métricas:

* **AUC-ROC:** mide la capacidad discriminativa del modelo para separar “default” de “no default”.
* **Gini coefficient:** derivado del AUC, muy usado en la industria bancaria para evaluar modelos de scoring.
* **KS Statistic (Kolmogorov–Smirnov):** mide la separación entre las distribuciones de score de clientes buenos y malos.
* **Precision-Recall (AUC-PR):** útil cuando hay desbalance (pocos clientes en default).
* **Matriz de confusión y métricas asociadas:**

  * **Recall (sensibilidad):** minimizar falsos negativos (detectar la mayoría de los defaulters).
  * **Precision:** evitar marcar como defaulters a demasiados buenos clientes.
* **Métricas de negocio/costo:** cada banco puede definir un “expected loss” o función de costos que penaliza más los falsos negativos que los falsos positivos.

---
